In [36]:
#importing the required dependencies,numpy for  linear algebra,pandas for working with data and matplotlib for visualizations.
import numpy as np
import pandas as pd

In [37]:
data = pd.read_csv('MNIST_data/weight-height.csv') #load the MNIST data.

In [38]:
data.head() #Getting an overview of how the dataframe looks.

,Gender,Height,Weight
0,Male,73.847017,241.893563
1,Male,68.781904,162.310473
2,Male,74.110105,212.740856
3,Male,71.730978,220.042470
4,Male,69.881796,206.349801


In [39]:
data.tail()

,Gender,Height,Weight
9995,Female,66.172652,136.777454
9996,Female,67.067155,170.867906
9997,Female,63.867992,128.475319
9998,Female,69.034243,163.852461
9999,Female,61.944246,113.649103


In [40]:
data.shape

(10000, 3)

In [41]:
data.describe()

,Height,Weight
count,10000.000000,10000.000000
mean,66.367560,161.440357
std,3.847528,32.108439
min,54.263133,64.700127
25%,63.505620,135.818051
50%,66.318070,161.212928
75%,69.174262,187.169525
max,78.998742,269.989699


In [42]:
data.nunique()

Gender        2
Height    10000
Weight    10000
dtype: int64

In [43]:
data.dtypes

Gender     object
Height    float64
Weight    float64
dtype: object

In [44]:
data.isnull().sum()

Gender    0
Height    0
Weight    0
dtype: int64

In [45]:
data['Gender'].value_counts()

Male      5000
Female    5000
Name: Gender, dtype: int64

In [46]:
data.drop_duplicates(keep='first',inplace=True)
print(data.shape)

(10000, 3)


In [47]:
data.columns
data['Gender'].replace(to_replace="Male",value=1,inplace=True)
data['Gender'].replace(to_replace="Female",value=0,inplace=True)

In [48]:
data.head()

,Gender,Height,Weight
0,1,73.847017,241.893563
1,1,68.781904,162.310473
2,1,74.110105,212.740856
3,1,71.730978,220.042470
4,1,69.881796,206.349801


In [49]:
for col in ['Gender']:
    data[col] = data[col].astype('category')
for col in ['Height']:
    data[col] = data[col].astype('int')
for col in ['Weight']:
    data[col] = data[col].astype('int')

In [50]:
data.dtypes

Gender    category
Height       int32
Weight       int32
dtype: object

In [51]:
data = np.array(data) #passsing the new numpyfied data to the new data variable.

In [52]:
m,n=data.shape # mxn RowxCol.

In [53]:
print(m,n) #m denoting the total data and n denoting the pixels per data.

10000 3


In [54]:
data

array([[  1,  73, 241],
       [  1,  68, 162],
       [  1,  74, 212],
       ...,
       [  0,  63, 128],
       [  0,  69, 163],
       [  0,  61, 113]], dtype=int64)

In [55]:
data.T

array([[  1,   1,   1, ...,   0,   0,   0],
       [ 73,  68,  74, ...,  63,  69,  61],
       [241, 162, 212, ..., 128, 163, 113]], dtype=int64)

In [56]:
np.random.shuffle(data) # shuffle before splitting into dev and training sets.

#This is the Cross-Validation split
data_dev = data[0:1000].T #Transposing the data from row vector to coloumn vector.
Y_dev = data_dev[0] #Now this would be the first Row(the output/the label/the aim).
X_dev = data_dev[1:n] #Now this is the data part after seperating the label hence the 1:n(n from the dimension).
X_dev = X_dev / 255. #Normalizing the values.

#This is the Train Split
data_train = data[1000:m].T #Transposing the data from row vector to coloumn vector.
Y_train = data_train[0] #Now this would be the first Row(the output/the label/the aim).
X_train = data_train[1:n] #Now this is the data part after seperating the label hence the 1:n(n from the dimension).
X_train = X_train / 255. #Normalization
_,m_train = X_train.shape

In [57]:
print(f"The Y_Train data:{Y_train}")
print(f"The X_train data:{X_train[0]}") #Just checking the 0th pos as its a big array.
print(f"The X_train data shape:{X_train[:,0].shape}") #Checking the shape to see the total pixel points are there.

The Y_Train data:[1 1 0 ... 1 0 1]
The X_train data:[0.27843137 0.28235294 0.22745098 ... 0.25882353 0.25098039 0.25490196]
The X_train data shape:(2,)


In [58]:
Y_train

array([1, 1, 0, ..., 1, 0, 1], dtype=int64)

In [59]:
a,b=X_train.shape
X_train.shape

(2, 9000)

We need to run gradient descent 

first we need to initialize some parameters.

In [60]:
def init_params(): #1
    W1 = np.random.randn(10,2) 
    W2 = np.random.randn(2,10)
    b1 = np.random.rand(10, 1) - 0.5 #(1,n_neurons)
    b2 = np.random.rand(1,1) - 0.5
    return W1, b1, W2, b2

In [61]:
#Needed in the forward Propogation
def ReLU(Z): #3
    return np.maximum(Z, 0) #Just selecting the max function.

In [62]:
def forward_prop(W1, b1, W2, b2, X): #2
    Z1 = W1.dot(X) + b1 #Z1 is the non-active set of neurons.X is the input as its the input layer.
    A1 = ReLU(Z1) #A1 is the activation function to pass the values
    Z2 = W2.dot(A1) + b2 #Z2 its the second layer,which takes the dot product from A1 ie;prev output from neuron.
    A2 = softmax(Z2) #A2 has softmax applied data for further classification.
    return Z1, A1, Z2, A2


In [63]:
#Needed in the BackPropogation to undo the applied functions.
def ReLU_deriv(Z):#7
    return Z > 0

In [64]:
#for backward propogation
def one_hot(Y): #6
    one_hot_Y = np.zeros((Y.size, Y.max() + 1)) #To create an empty matrix(filled with 0)
    one_hot_Y[np.arange(Y.size), Y] = 1 #Setting the respective values as needed.
    one_hot_Y = one_hot_Y.T #Transposing the matrix
    return one_hot_Y


In [65]:
one_hot(Y_train).shape

(2, 9000)

In [66]:
def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y): #5
    one_hot_Y = one_hot(Y)
    m=Y.size
    dZ2 = A2 - one_hot_Y #To find the error.
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1) #Revert changes.
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    
    return dW1, db1, dW2, db2

In [67]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha): #Alpha would act as the learning parameter.
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2


In [68]:
def get_predictions(A2):
    return np.argmax(A2, 0) #on the Xaxis


In [69]:
def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

In [70]:
#Needed in the forward Propogation
def softmax(Z): #4
    A = np.exp(Z) / sum(np.exp(Z))
    return A

In [71]:
def gradient_descent(X, Y, alpha, iterations):
    
    W1, b1, W2, b2 = init_params()
    
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0: #setting the denoter to occur every 10 steps.
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [74]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 1000)

Iteration:  0
[1 1 1 ... 1 1 1] [1 1 0 ... 1 0 1]
0.5002222222222222
Iteration:  10
[1 1 0 ... 1 1 1] [1 1 0 ... 1 0 1]
0.6686666666666666
Iteration:  20
[1 1 0 ... 1 1 1] [1 1 0 ... 1 0 1]
0.6928888888888889
Iteration:  30
[1 1 0 ... 1 1 1] [1 1 0 ... 1 0 1]
0.7053333333333334
Iteration:  40
[1 1 0 ... 1 1 1] [1 1 0 ... 1 0 1]
0.7171111111111111
Iteration:  50
[1 1 0 ... 1 1 1] [1 1 0 ... 1 0 1]
0.7226666666666667
Iteration:  60
[1 1 0 ... 1 1 1] [1 1 0 ... 1 0 1]
0.7302222222222222
Iteration:  70
[1 1 0 ... 1 1 1] [1 1 0 ... 1 0 1]
0.7391111111111112
Iteration:  80
[1 1 0 ... 1 1 1] [1 1 0 ... 1 0 1]
0.7503333333333333
Iteration:  90
[1 1 0 ... 1 1 1] [1 1 0 ... 1 0 1]
0.7516666666666667
Iteration:  100
[1 1 0 ... 1 1 1] [1 1 0 ... 1 0 1]
0.7602222222222222
Iteration:  110
[1 1 0 ... 1 1 1] [1 1 0 ... 1 0 1]
0.7675555555555555
Iteration:  120
[1 1 0 ... 1 1 1] [1 1 0 ... 1 0 1]
0.7693333333333333
Iteration:  130
[1 1 0 ... 1 1 1] [1 1 0 ... 1 0 1]
0.7746666666666666
Iteration:  140
[

In [75]:
print("The Captured weights/biases are")
print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
print(W1)
print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
print(W2)
print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
print(b1)
print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
print(b1)

The Captured weights/biases are
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
[[ 1.35210615 -0.62078576]
 [ 1.34416937  3.29270088]
 [-0.77519343 -2.49798149]
 [-0.29695302 -1.61464403]
 [-0.04464896  2.35957023]
 [ 0.29786261 -0.17317509]
 [-1.38349413  0.08405886]
 [ 0.014589    2.59663355]
 [-2.14012079 -0.26000968]
 [ 1.35972363 -1.14692273]]
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
[[ 1.51637423  1.3758062   1.33145586  1.13158079 -2.08660606  0.15627939
  -0.30701688 -2.19528159 -0.08325524  0.29815863]
 [-0.43666682 -0.89230797 -0.91504185 -0.01625231  4.34382249 -0.01565936
  -0.23059385  2.43640205  0.87393039 -1.03093919]]
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
[[-0.81570779]
 [-0.8842642 ]
 [-0.89158254]
 [-0.40083765]
 [-1.16069818]
 [-0.5998369 ]
 [-0.47529718]
 [-1.30663804]
 [-1.1999505 ]
 [-0.87323339]]
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
[[-0.81570779]
 [-0.8842642 ]
 [-0.89158254]
 [-0.40083765]
 [-1.16069818]
 [-0.5998369

In [76]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions


In [77]:
def test_prediction(index, W1, b1, W2, b2):
    prediction = make_predictions(X_dev[:, index, None], W1, b1, W2, b2)
    label = Y_dev[index]
    #Setting up the test files.
    print("Prediction: ", prediction)
    print("Label: ", label)
   

In [78]:
test_prediction(2, W1, b1, W2, b2)

Prediction:  [1]
Label:  1


In [79]:
test_prediction(10, W1, b1, W2, b2)

Prediction:  [0]
Label:  0


In [80]:
test_prediction(3, W1, b1, W2, b2)

Prediction:  [1]
Label:  0


In [81]:
test_prediction(0, W1, b1, W2, b2)

Prediction:  [1]
Label:  1


In [82]:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
acc=get_accuracy(dev_predictions, Y_dev)
print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
print("The Neural Network has an accuracy of :",acc*100,"%")

[1 1 1 1 1 0 0 0 1 0 0 1 0 0 1 0 1 0 1 1 1 1 1 1 0 0 1 1 1 0 1 0 1 1 0 1 1
 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 1 1 0 0 1 1 0 1 1 1 1 0 0 1 0 0 1 0 1 0 1 1
 1 1 0 0 1 0 1 0 1 0 0 1 1 1 0 1 1 0 1 0 1 0 0 0 1 0 1 1 0 1 0 0 1 0 0 0 1
 1 1 0 1 0 1 1 1 0 1 1 0 0 1 1 1 1 1 0 1 0 0 0 0 1 1 0 0 1 1 1 1 1 1 0 1 1
 0 0 0 0 1 1 0 0 0 0 1 0 1 0 1 0 1 1 1 1 1 0 0 1 1 1 0 0 1 0 1 0 0 0 1 1 1
 0 0 0 1 0 1 0 0 1 0 1 1 0 0 1 0 0 1 0 1 1 1 0 0 0 1 0 1 0 1 1 1 0 0 0 0 0
 1 1 0 0 0 1 0 0 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 1 1 1 0 0 1 0 0 1 1
 1 1 0 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 0 0 1 0 0 1 1 0 0 1 1 1
 1 1 1 0 0 1 0 1 0 0 0 1 0 1 1 0 0 1 0 0 0 1 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1
 1 0 1 1 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 1 0 0 0 1 0 1 1 1 1
 1 1 0 0 1 0 1 1 1 0 0 0 1 0 1 1 1 1 0 1 1 0 0 1 1 1 1 1 0 0 0 1 1 1 1 0 1
 0 1 0 1 1 0 0 0 0 1 1 1 0 0 1 1 1 1 0 1 0 0 1 1 0 1 0 1 0 1 1 0 0 1 1 1 0
 1 1 1 0 0 1 0 0 0 1 0 1 1 0 1 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0
 0 0 1 1 1 0 0 0 1 0 1 0 